<a href="https://colab.research.google.com/github/nhattan206/AI_CUOIKY/blob/main/AI_CuoiKy_App_RealTime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tkinter import *
from tkinter.filedialog import Open, SaveAs
import numpy as np
import cv2 as cv
from keras.models import load_model
from keras.models import Input, Model

model = load_model('CuoiKy.h5')

def detect_points(face_img):
    me  = np.array(face_img)/255
    x_test = np.expand_dims(me, axis=0)
    x_test = np.expand_dims(x_test, axis=3)

    y_test = model.predict(x_test)
    label_points = (np.squeeze(y_test)*48)+48 
    
    return label_points
# Load haarcascade
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
dimensions = (96, 96)

class Main(Frame):
    def __init__(self, parent):
        Frame.__init__(self, parent)
        self.parent = parent
        self.initUI()
  
    def initUI(self):
        self.parent.title("Nhận diện các bộ phận trên khuôn mặt")
        self.pack(fill=BOTH, expand=1)
  
        menubar = Menu(self.parent)
        self.parent.config(menu=menubar)
  
        fileMenu = Menu(menubar)
        fileMenu.add_command(label="Mở Camera", command=self.onOpenCam)
        fileMenu.add_command(label="Nhận diện", command=self.onRecognition)
        fileMenu.add_separator()
        fileMenu.add_command(label="Thoát", command=self.quit)
        menubar.add_cascade(label="File", menu=fileMenu)
        #self.txt = Text(self)
        #self.txt.pack(fill=BOTH, expand=1)
  
    def onOpenCam(self):
        # Đường dẫn video hoặc webcam (số 0)
        cap = cv.VideoCapture('video2.mp4')
        while 1:
            ret, frame = cap.read()
            if not ret:
                print('unavailable')
            # cv.imwrite('data.jpg',frame)
            # img = cv.imread('data.jpg')

            default_img = frame
            gray_img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_img, 1.3, 5)
            #faces = face_cascade.detectMultiScale(gray_img, 4, 6)

            faces_img = np.copy(gray_img)

            # plt.rcParams["axes.grid"] = False


            all_x_cords = []
            all_y_cords = []

            for i, (x,y,w,h) in enumerate(faces):
                
                h += 10
                w += 10
                x -= 5
                y -= 5
                
                just_face = cv.resize(gray_img[y:y+h,x:x+w], dimensions)
                cv.rectangle(faces_img,(x,y),(x+w,y+h),(255,0,0),1)
                
                scale_val_x = w/96
                scale_val_y = h/96
                
                label_point = detect_points(just_face)
                all_x_cords.append((label_point[::2]*scale_val_x)+x)
                all_y_cords.append((label_point[1::2]*scale_val_y)+y)
            
                # Vẽ các hình vuông với tọa độ góc là các điểm chỉ định
                cv.rectangle(default_img,(int(label_point[4]*scale_val_x+x),int(label_point[5]*scale_val_y+y)),(int((label_point[4]+16)*scale_val_x+x),int((label_point[5]+10)*scale_val_y+y)),(0,255,255),2)
                cv.putText(default_img,'mat phai',(int(label_point[4]*scale_val_x+x-10),int(label_point[5]*scale_val_y+y)),cv.FONT_HERSHEY_PLAIN,1,(0,0,255),1)
                cv.rectangle(default_img,(int(label_point[10]*scale_val_x+x),int(label_point[11]*scale_val_y+y)),(int((label_point[10]+16)*scale_val_x+x),int((label_point[11]+10)*scale_val_y+y)),(0,255,255),2)
                cv.putText(default_img,'mat trai',(int(label_point[10]*scale_val_x+x-10),int(label_point[11]*scale_val_y+y)),cv.FONT_HERSHEY_PLAIN,1,(0,0,255),1)
                cv.circle(default_img,(int(label_point[20]*scale_val_x+x),int(label_point[21]*scale_val_y+y)),20,(200,100,50),2)
                cv.putText(default_img,'mui',(int(label_point[20]*scale_val_x+x-10),int(label_point[21]*scale_val_y+y)),cv.FONT_HERSHEY_PLAIN,1,(200,100,50),1)
                cv.rectangle(default_img,(int(label_point[24]*scale_val_x+x),int(label_point[25]*scale_val_y+y)),(int((label_point[24]+40)*scale_val_x+x),int((label_point[25]+10)*scale_val_y+y)),(209,206,0),2)
                cv.putText(default_img,'mieng',(int(label_point[24]*scale_val_x+x-10),int(label_point[25]*scale_val_y+y)),cv.FONT_HERSHEY_PLAIN,1,(209,206,0),1)
            cv.namedWindow('RealTime Test', cv.WINDOW_AUTOSIZE)
            cv.imshow('RealTime Test',default_img)
            if cv.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                cv.destroyAllWindows()
                break
    def onRecognition(self):
        None

root = Tk()
Main(root)
root.geometry("480x480+100+100")
root.mainloop()
